[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/neurogym/neurogym/blob/main/docs/examples/annubes.ipynb)


## `AnnubesEnv` environment

This notebook is a simple example of how to use the `AnnubesEnv` class to create a custom environment and use it to
train a reinforcement learning agent with `stable_baselines3`.


### Installation

**Google Colab:** Uncomment and execute cell below when running this notebook on google colab.

**Local:** Follow [these instructions](https://github.com/neurogym/neurogym?tab=readme-ov-file#installation) when running
this notebook locally.


In [ ]:
# ! pip install neurogym[rl]

Import everything that we are going to need below.

In [ ]:
import warnings
warnings.filterwarnings("default")

from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_checker import check_env

import neurogym as ngym
from neurogym.envs.native.annubes import AnnubesEnv

Let's create an environment, check it works and visualize it.


In [ ]:
env = AnnubesEnv()

# check the custom environment and output additional warnings (if any)
check_env(env)

# check the environment with a random agent
obs, info = env.reset()
n_steps = 10
for _ in range(n_steps):
    # random action
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated:
        obs, info = env.reset()

print(env.timing)
print("----------------")
print(env.observation_space)
print(env.observation_space.name)
print("----------------")
print(env.action_space)
print(env.action_space.name)

In [ ]:
fig = ngym.utils.plot_env(env, ob_traces=["fixation", "start", "a", "v"], num_trials=10)

The `session` argument can also contain combinations of stimuli that should be presented _together_ with a certain probability. Any such combinations can be specified as tuples in the `session` dictionary with the desired probability. For instance, in the following example, modalities `v` and `a` will appear together about 40% of the time.

In [ ]:
session = {"v": 1.0, "a": 0.0}
env = AnnubesEnv(session=session)
env.session

In [ ]:
fig = ngym.utils.plot_env(
    env,
    ob_traces=["fixation", "start", "a", "v"],
    num_trials=30,
)

Combinations of stimuli can even be specified without necessarily adding the individual stimuli to the `session` dictionary. The environment will automatically add missing elementary modalities to the `session` dictionary with a probability set to 0.0. For instance, the following environment will present both `a` and `v` 50% of the non-catch trials and only `v` for the other 50%. The missing `a` modality will be added automatically to the session dictionary with a probability set to 0.

In [ ]:
session = {"v": 0.5, ("v", "a"): 0.5}
env = AnnubesEnv(session=session)
env.session

This makes it easy, for instance, to instantiate an environment where two modalities are presented together 100% of the time but never in isolation:

In [ ]:
session = {("v", "a"): 1}
env = AnnubesEnv(session=session)
env.session

In [ ]:
fig = ngym.utils.plot_env(
    env,
    ob_traces=["fixation", "start", "a", "v"],
    num_trials=30,
)

Another point to consider is that the probabilities in the `session` dictionary don't have to add up to 1 since they are normalised upon instantiation. As a side effect if you specify a single entry in the `session` dictionary, that entry will appear in 100% of the non-catch trials, regardless of the probability assigned (as long as it is not 0) since the probability of that entry will end up being normalised to 1, as illustrated below:

In [ ]:
# It doesn't matter what probability we set for 'v', as long as it is not 0
session1 = {"v": 0.5}
session2 = {"v": 0.1}
env1 = AnnubesEnv(session=session1)
env2 = AnnubesEnv(session=session2)
env1.session, env2.session

On that note, an error is raised if the probabilities in the `session` dictionary add up to 0:

In [ ]:
try:
    session = {"v": 0.0, "a": 0.0}
    env = AnnubesEnv(session=session)
except ValueError as e:
    print(f"Expected error raised: {e}")

As a final remark, it is also possible to restrict the number of sequential presentations of any modality by using the `max_sequential` argument. It can be an integer or a dictionary, where an integer is interpreted as a global limit applying to all modalities, whereas a dictionary gives more fine-grained control:

In [ ]:
session = {"v": 0.5, "a": 0.5}
max_sequential = 2
env = AnnubesEnv(session=session, max_sequential=max_sequential)
env.session

In [ ]:
fig = ngym.utils.plot_env(
    env,
    ob_traces=["fixation", "start", "a", "v"],
    num_trials=30,
)

In [ ]:
session = {"v": 0.5, "a": 0.5}
max_sequential = {"v": 1, "a": 3}
env = AnnubesEnv(session=session, max_sequential=max_sequential)
env.session

In [ ]:
fig = ngym.utils.plot_env(
    env,
    ob_traces=["fixation", "start", "a", "v"],
    num_trials=30,
)

The environment also checks if the requested `max_sequential` condition is satisfiable. For instance, an error is raised if the probability of a catch trial is 0 and the probability of one or more modalities is 1 but with an imposed `max_sequential` limit:

In [ ]:
try:
    # Here, the probability of 'v' is in fact 1 because it appears in all entries of the session dictionary
    session = {"v": 0.5, ("a", "v"): 0.5}

    # Impose a global limit on sequential presentations of the same stimulus.
    max_sequential = 5

    # Ensure that there are no catch trials
    catch_prob = 0.0
    env = AnnubesEnv(session=session, catch_prob = catch_prob, max_sequential=max_sequential)
except ValueError as e:
    print(f"Expected error raised: {e}")

Removing the limit eliminates the error:

In [ ]:
# Here, the probability of 'v' is in fact 1 because it appears in all entries of the session dictionary
session = {"v": 0.5, ("a", "v"): 0.5}

# Ensure that there are no catch trials
catch_prob = 0.0
env = AnnubesEnv(
    session=session, catch_prob=catch_prob
)

# This works...
env.session

In [ ]:
fig = ngym.utils.plot_env(
    env,
    ob_traces=["fixation", "start", "a", "v"],
    num_trials=30,
)

## Training `AnnubesEnv`

### 1. Regular training

We can train `AnnubesEnv` using one of the models defined in `stable_baselines3`, for example [`A2C`](https://stable-baselines3.readthedocs.io/en/master/modules/a2c.html).


In [ ]:
# Instantiate a default environment and train an agent on it.
env = AnnubesEnv()
env_vec = DummyVecEnv([lambda: env])
model = A2C("MlpPolicy", env_vec, verbose=0)
model.learn(total_timesteps=20000, log_interval=1000)
env_vec.close()

# Plot example trials with the trained agent
data = ngym.utils.plot_env(
    env, num_trials=10, ob_traces=["fixation", "start", "a", "v"], model=model
)

### 2. Sequential training

We can also train `AnnubesEnv` using a sequential training approach. This is useful when we want to train the agent in multiple stages, each with a different environment configuration. This can be useful for:

- **Curriculum learning**: Gradually increase the difficulty of the environments. Start with simpler tasks and progressively move to more complex ones, allowing the agent to build on its previous experiences.

- **Domain randomization**: Vary the environment dynamics (e.g., physics, obstacles) during training to improve the agent's robustness to changes in the environment.

- **Transfer learning**: If you have access to different agents or architectures, you can use transfer learning techniques to fine-tune the model on a new environment.

In this case it is important to include all the possible observations in each environment, even if not all of them are used. This is because the model is initialized with the first environment's observation space and it is not possible to change it later.


In [ ]:
env1 = AnnubesEnv({"v": 1, "a": 0})
env1_vec = DummyVecEnv([lambda: env1])

# Create a model and train it with the first environment
model = A2C("MlpPolicy", env1_vec, verbose=0)
model.learn(total_timesteps=10000)
env1_vec.close()

# Plot example trials with the trained agent
data = ngym.utils.plot_env(
    env1,
    num_trials=10,
    ob_traces=["fixation", "start", "a", "v"],
    model=model,
)

In [ ]:
# Switch to the second environment and continue training
env2 = AnnubesEnv({"v": 0, "a": 1})
env2_vec = DummyVecEnv([lambda: env2])

# Set the model's environment to the new environment
model.set_env(env2_vec)
model.learn(total_timesteps=10000)
env2_vec.close()

# Plot example trials with the trained agent
data = ngym.utils.plot_env(
    env2,
    num_trials=10,
    ob_traces=["fixation", "start", "a", "v"],
    model=model,
)

In [ ]:
# Switch to the third environment and finish training
env3 = AnnubesEnv({"v": 0.5, "a": 0.5})
env3_vec = DummyVecEnv([lambda: env3])

# Set the model's environment to the new environment
model.set_env(env3_vec)
model.learn(total_timesteps=20000)
env3_vec.close()

# Plot example trials with the trained agent
data = ngym.utils.plot_env(
    env3, num_trials=10, ob_traces=["fixation", "start", "a", "v"], model=model
)

In [ ]:
# Save the final model after all training
model.save("final_model")